In [1]:
import re
import numpy as np
import pandas as pd

DATA_PATH = "../Gyrate_UserResults.xlsx"
KEEP_ATTEMPT = "first"   # options: "first", "latest", "best"

def parse_failed_levels(s: str | float | None) -> dict[int, int]:
    """
    Parse the 'FailedLevels' column into {level -> total_failures_at_that_level}.
    Each token looks like 'A:B:C'. We keep max(B) per A (usually equals the count).
    """
    fails: dict[int, int] = {}
    if pd.isna(s):
        return fails
    s = str(s).strip()
    if not s:
        return fails
    for tok in s.split(","):
        parts = tok.strip().split(":")
        if len(parts) < 2:
            continue
        a, b = parts[0].strip(), parts[1].strip()
        if a.isdigit() and b.isdigit():
            A = int(a)
            B = int(b)
            fails[A] = max(fails.get(A, 0), B)
    return fails

# test
parse_failed_levels("1:1:0,9:1:6,9:2:4,10:1:6,11:1:1,11:2:6,11:3:5,12:1:0,12:2:6,12:3:5")

df = pd.read_excel(DATA_PATH)
df

,AccountId,AssessmentId,AssessmentVersionId,Score,DisplayScore,Percentage,Percentile,Locale,CreationDate,TotalResponses,CorrectResponses,ReactionTime,BonusAwards,Level,FailedLevels,EarlyResponses,LateResponses
0,595780,12,151,10,1000,47.619048,0,en,2025-10-14 10:33:20.686,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,595780,12,151,10,1000,47.619048,0,en,2025-10-14 10:33:20.686,NaN,NaN,5.124400,NaN,NaN,NaN,NaN,NaN
2,595780,12,151,10,1000,47.619048,0,en,2025-10-14 10:33:20.686,20.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN
3,595780,12,151,10,1000,47.619048,0,en,2025-10-14 10:33:20.686,NaN,NaN,NaN,0.0,10.0,"1:1:0,9:1:6,9:2:4,10:1:6,11:1:1,11:2:6,11:3:5,...",NaN,NaN
4,694859,12,151,9,900,42.857143,0,nl,2025-10-14 10:27:23.136,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,687820,12,151,6,500,28.571429,0,en,2025-09-26 01:44:14.802,NaN,NaN,NaN,0.0,6.0,"5:1:0,5:2:3,5:3:4,6:1:4,6:2:1,7:1:2,7:2:2,7:3:...",NaN,NaN
2996,687811,12,151,21,2000,100.000000,0,es,2025-09-26 01:42:21.687,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2997,687811,12,151,21,2000,100.000000,0,es,2025-09-26 01:42:21.687,NaN,NaN,9.645667,NaN,NaN,NaN,NaN,NaN
2998,687811,12,151,21,2000,100.000000,0,es,2025-09-26 01:42:21.687,30.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
def load_sessions_dev(path=DATA_PATH, keep_attempt=KEEP_ATTEMPT):
    """
    Read the raw summary rows and coalesce each group of duplicate-key rows into one row
    by taking the first non-null value inside each group for the data columns.
    """
    df = pd.read_excel(path)
    df.columns = [c.strip() for c in df.columns]

    # Drop unused columns if present
    df = df.drop(columns=[c for c in ["Percentile", "BonusAwards", "EarlyResponses", "LateResponses"] if c in df.columns],
                 errors="ignore")

    # Types we rely on
    for c in ["Level", "CorrectResponses", "TotalResponses", "ReactionTime"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    if "CreationDate" in df.columns:
        df["CreationDate"] = pd.to_datetime(df["CreationDate"], errors="coerce", infer_datetime_format=True)

    # Columns that define identical groups (these are equal within each 4-row block)
    key_cols = [
        "AccountId", "AssessmentId", "AssessmentVersionId", "Score",
        "DisplayScore", "Percentage", "Locale", "CreationDate"
    ]
    key_cols = [c for c in key_cols if c in df.columns]  # guard

    # Columns where only one row in the block has the value; pick the first non-null
    value_cols = [
        "TotalResponses", "CorrectResponses", "ReactionTime", "Level", "FailedLevels"
    ]
    value_cols = [c for c in value_cols if c in df.columns]  # guard

    # Helper: first non-null in the group
    def first_valid(s):
        s = s.dropna()
        return s.iloc[0] if not s.empty else np.nan

    # Group by the invariant columns and coalesce value columns
    if key_cols:
        agg_map = {c: first_valid for c in value_cols}
        collapsed = (
            df.groupby(key_cols, as_index=False)
              .agg(agg_map)  # returns one row per unique key
        )
    else:
        # Fallback if keys missing: just take first non-null per entire df (unlikely)
        collapsed = df[value_cols].agg(first_valid).to_frame().T

    return collapsed


def remove_repeated_attempts(df,  keep_attempt=KEEP_ATTEMPT): 

    """
    Read the raw summary rows and keep ONE row per AccountId.
    - 'first'  : earliest CreationDate per AccountId
    - 'latest' : latest CreationDate per AccountId
    - 'best'   : highest Level, tie-broken by latest time
    """

    if keep_attempt == "first":
        df = (df.sort_values(["AccountId", "CreationDate"], ascending=[True, True])
                .drop_duplicates(subset=["AccountId"], keep="first"))
    elif keep_attempt == "latest":
        df = (df.sort_values(["AccountId", "CreationDate"], ascending=[True, False])
                .drop_duplicates(subset=["AccountId"], keep="first"))
    elif keep_attempt == "best":
        df = (df.sort_values(["AccountId", "Level", "CreationDate"],
                             ascending=[True, False, False])
                .drop_duplicates(subset=["AccountId"], keep="first"))
    else:
        raise ValueError("KEEP_ATTEMPT must be 'first', 'latest', or 'best'.")

    return df




In [3]:
df = load_sessions_dev(DATA_PATH, KEEP_ATTEMPT)
df

/var/folders/ww/rbvxp9wd1r18p36rblrbsz0r0000gp/T/ipykernel_2104/4148836556.py:18: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["CreationDate"] = pd.to_datetime(df["CreationDate"], errors="coerce", infer_datetime_format=True)


,AccountId,AssessmentId,AssessmentVersionId,Score,DisplayScore,Percentage,Locale,CreationDate,TotalResponses,CorrectResponses,ReactionTime,Level,FailedLevels
0,130850,12,151,8,800,38.095238,en,2025-10-13 17:02:37.326,18.0,8.0,1.687625,8.0,"1:1:0,7:1:2,7:2:5,8:1:4,9:1:0,9:2:4,9:3:7,10:1..."
1,163974,12,151,18,1500,85.714286,pt-BR,2025-10-13 15:35:38.542,36.0,15.0,14.453750,18.0,"6:1:0,6:2:5,6:3:5,7:1:5,12:1:5,12:2:2,14:1:0,1..."
2,168052,12,151,6,600,28.571429,nl,2025-09-29 12:14:35.498,13.0,6.0,2.271600,6.0,"6:1:5,7:1:4,7:2:3,7:3:0,8:1:6,8:2:2,8:3:4"
3,227411,12,151,21,1800,100.000000,es-MX,2025-09-27 12:43:00.126,36.0,18.0,17.825389,21.0,"1:1:1,7:1:4,7:2:4,7:3:5,8:1:3,8:2:1,13:1:0,15:..."
4,258059,12,151,8,800,38.095238,es-MX,2025-10-10 20:32:58.831,18.0,8.0,2.178375,8.0,"1:1:0,7:1:5,7:2:2,8:1:2,9:1:7,9:2:7,9:3:2,10:1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,694630,12,151,8,800,38.095238,es,2025-10-14 00:54:07.416,14.0,8.0,2.044714,8.0,"9:1:7,9:2:1,9:3:2,10:1:3,10:2:5,10:3:5"
746,694645,12,151,6,600,28.571429,en,2025-10-14 01:45:03.021,15.0,6.0,2.759800,6.0,"4:1:2,6:1:5,6:2:3,7:1:1,7:2:2,7:3:0,8:1:4,8:2:..."
747,694710,12,151,6,500,28.571429,en,2025-10-14 08:43:19.184,16.0,5.0,2.939500,6.0,"4:1:3,4:2:3,4:3:2,5:1:3,6:1:4,7:1:2,7:2:2,7:3:..."
748,694780,12,151,7,700,33.333333,en,2025-10-14 10:09:11.314,19.0,7.0,3.358000,7.0,"4:1:3,4:2:3,6:1:5,6:2:5,7:1:5,7:2:4,8:1:3,8:2:..."


In [4]:
print(len(set(df['AccountId'].tolist())))
df = remove_repeated_attempts(df, KEEP_ATTEMPT)
df

702


,AccountId,AssessmentId,AssessmentVersionId,Score,DisplayScore,Percentage,Locale,CreationDate,TotalResponses,CorrectResponses,ReactionTime,Level,FailedLevels
0,130850,12,151,8,800,38.095238,en,2025-10-13 17:02:37.326,18.0,8.0,1.687625,8.0,"1:1:0,7:1:2,7:2:5,8:1:4,9:1:0,9:2:4,9:3:7,10:1..."
1,163974,12,151,18,1500,85.714286,pt-BR,2025-10-13 15:35:38.542,36.0,15.0,14.453750,18.0,"6:1:0,6:2:5,6:3:5,7:1:5,12:1:5,12:2:2,14:1:0,1..."
2,168052,12,151,6,600,28.571429,nl,2025-09-29 12:14:35.498,13.0,6.0,2.271600,6.0,"6:1:5,7:1:4,7:2:3,7:3:0,8:1:6,8:2:2,8:3:4"
3,227411,12,151,21,1800,100.000000,es-MX,2025-09-27 12:43:00.126,36.0,18.0,17.825389,21.0,"1:1:1,7:1:4,7:2:4,7:3:5,8:1:3,8:2:1,13:1:0,15:..."
4,258059,12,151,8,800,38.095238,es-MX,2025-10-10 20:32:58.831,18.0,8.0,2.178375,8.0,"1:1:0,7:1:5,7:2:2,8:1:2,9:1:7,9:2:7,9:3:2,10:1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,694630,12,151,8,800,38.095238,es,2025-10-14 00:54:07.416,14.0,8.0,2.044714,8.0,"9:1:7,9:2:1,9:3:2,10:1:3,10:2:5,10:3:5"
746,694645,12,151,6,600,28.571429,en,2025-10-14 01:45:03.021,15.0,6.0,2.759800,6.0,"4:1:2,6:1:5,6:2:3,7:1:1,7:2:2,7:3:0,8:1:4,8:2:..."
747,694710,12,151,6,500,28.571429,en,2025-10-14 08:43:19.184,16.0,5.0,2.939500,6.0,"4:1:3,4:2:3,4:3:2,5:1:3,6:1:4,7:1:2,7:2:2,7:3:..."
748,694780,12,151,7,700,33.333333,en,2025-10-14 10:09:11.314,19.0,7.0,3.358000,7.0,"4:1:3,4:2:3,6:1:5,6:2:5,7:1:5,7:2:4,8:1:3,8:2:..."


In [5]:
def extract_columns(df: pd.DataFrame) -> pd.DataFrame:
    required_cols = ["AccountId", "Score", "Percentage"]
    
    # Ensure all columns exist
    missing = [col for col in required_cols if col not in df.columns]
    if missing:
        raise KeyError(f"Missing required columns: {missing}")
    
    return df[required_cols]
df = extract_columns(df)
df

,AccountId,Score,Percentage
0,130850,8,38.095238
1,163974,18,85.714286
2,168052,6,28.571429
3,227411,21,100.000000
4,258059,8,38.095238
...,...,...,...
745,694630,8,38.095238
746,694645,6,28.571429
747,694710,6,28.571429
748,694780,7,33.333333
